In [18]:
import numpy as np
import cvxpy as cp
import heapdict
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import time

from pytransform3d.rotations import *

import open3d as o3d
import copy

In [19]:
def pc_to_np(open3d_data):
    return np.asarray(open3d_data.points)

def np_to_pc(np_data):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_data)
    return pcd

def np_to_tuple(data):
    return tuple(data.flatten())

def tuple_to_np(data):
    return (np.asarray(list(data))).reshape((4,4))

def rotation_matrix(euler):
    R = np.identity(4)
    R[0:3,0:3] = matrix_from_euler_xyz(euler)
    return R

def scale_matrix(euler):
    S = np.identity(4)
    S[0,0] = euler[0]
    S[1,1] = euler[1]
    S[2,2] = euler[2]
    return S

def translation_matrix(euler):
    T = np.identity(4)
    T[0,3] = euler[0]
    T[1,3] = euler[1]
    T[2,3] = euler[2]
    return T

In [20]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def icp(source, target, T_init, threshold=0.8):
    
    evaluation = o3d.registration.evaluate_registration(source, target, threshold, T_init)

    reg_p2p = o3d.registration.registration_icp(source, target, threshold, T_init, 
                                                o3d.registration.TransformationEstimationPointToPoint())
    
    return reg_p2p.transformation

In [21]:
def compute_dist(src,tgt,T):
    src_cpy = copy.deepcopy(src)
    src_t = src_cpy.transform(T)
    
    src_np = pc_to_np(src_t)
    tgt_np = pc_to_np(tgt)
    
    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(tgt_np)
    distances, indices = neigh.kneighbors(src_np, return_distance=True)
    distances = distances.ravel()
    indices = indices.ravel()
    
    numUnique = len(np.unique(indices))
    
    eps = 0.1
    sumAboveEps = np.sum(np.where(distances > eps, distances, 0))
    
    return distances, numUnique, sumAboveEps

In [39]:
src_ref = "data/model_bunny.txt"
tgt_ref = "data/generated_bunny.txt"

source = o3d.io.read_point_cloud(src_ref, format='xyz')
target = o3d.io.read_point_cloud(tgt_ref, format='xyz')


# APPLY RANDOM TRANSFORMATION TO TARGET


p2p = np.identity(4)

R = rotation_matrix(10*np.random.rand(3))
p2p = np.dot(R,p2p)

'''
ratio = 1.3
S = scale_matrix([ratio,ratio,ratio])
p2p = np.dot(S,p2p)

T = translation_matrix([0.1,0.2,-0.1])
p2p = np.dot(T,p2p)
'''

target.transform(p2p)




T_iden = np.identity(4)
draw_registration_result(source,target,T_iden)

In [ ]:

T_iden = np.identity(4)

distances, numUnique, sumAboveEps = compute_dist(source,target,T_iden)
print("Net distance: %.3f | Sum above eps: %.3f | Unique matchings: %d" % (np.sum(distances), sumAboveEps, numUnique ))


p2p = np.identity(4)


maxIters = 80
i = 0
prevSumAboveEps = 0

hd = heapdict.heapdict()
hd_size = 0



d = 3
pi = 3.14
for i in range(-d,d):
    for j in range(-d,d):
        for k in range(-d,d):
            R = rotation_matrix([2*pi*(i/d),2*pi*(j/d),2*pi*(k/d)])

            distances, numUnique, sumAboveEps = compute_dist(source, target, np.dot(R,p2p))
            hd[np_to_tuple(np.dot(R,p2p))] = sumAboveEps
            hd_size += 1



while(i < maxIters) and (hd_size > 0):
    
    p2p, prevSumAboveEps = hd.popitem()
    p2p = tuple_to_np(p2p)
    hd_size -= 1
    
    p2p = icp(source, target, p2p, threshold=0.8)
    distances, numUnique, sumAboveEps = compute_dist(source, target, p2p)
    
    print("%d Net distance: %.3f | Sum above eps: %.3f | Unique matchings: %d" % (i, np.sum(distances), sumAboveEps, numUnique ))

    if(sumAboveEps < 0.005):
        break
    
    '''
    if(abs(sumAboveEps - prevSumAboveEps) < 5.0):
        
        print('adding new rotations...')
        d = 2
        pi = 3.14
        for i in range(-d,d):
            for j in range(-d,d):
                for k in range(-d,d):
                    if not ( i == 0 and j == 0 and k == 0):
                        R = rotation_matrix([2*pi*(i/d),2*pi*(j/d),2*pi*(k/d)])

                        distances, numUnique, sumAboveEps = compute_dist(source, target, np.dot(R,p2p))
                        hd[np_to_tuple(np.dot(R,p2p))] = sumAboveEps
                        hd_size += 1
        
        print('done adding.')
    '''
        
    if(abs(sumAboveEps - prevSumAboveEps) > 5.0):
        hd[np_to_tuple(p2p)] = sumAboveEps
        hd_size += 1
    
    i += 1

print()
print(p2p)


Net distance: 4601.128 | Sum above eps: 3883.820 | Unique matchings: 3579
2 Net distance: 2182.484 | Sum above eps: 1103.280 | Unique matchings: 4785
3 Net distance: 2181.262 | Sum above eps: 1077.588 | Unique matchings: 4601
4 Net distance: 2137.831 | Sum above eps: 992.131 | Unique matchings: 4479
5 Net distance: 2050.916 | Sum above eps: 868.091 | Unique matchings: 4308
6 Net distance: 1948.403 | Sum above eps: 758.567 | Unique matchings: 4517
7 Net distance: 1729.679 | Sum above eps: 583.444 | Unique matchings: 5296
8 Net distance: 1667.151 | Sum above eps: 548.957 | Unique matchings: 5759
9 Net distance: 1652.863 | Sum above eps: 531.944 | Unique matchings: 5821
10 Net distance: 1640.097 | Sum above eps: 512.478 | Unique matchings: 5855
11 Net distance: 1640.014 | Sum above eps: 506.783 | Unique matchings: 5849
12 Net distance: 1640.020 | Sum above eps: 506.458 | Unique matchings: 5849
13 Net distance: 2182.500 | Sum above eps: 1103.501 | Unique matchings: 4788
14 Net distance: 21

In [38]:
R = rotation_matrix([0,0,0])


draw_registration_result(source,target, np.dot(R, p2p))

KeyboardInterrupt: 